# Metric Scores for Football Manager Roles and Positions
- inspired by https://www.youtube.com/watch?v=hnAuOakqR90&t=460s

In [61]:
import pandas as pd
import numpy as np

In [62]:
import glob
import os

In [63]:
directory_path = (os.path.join('/Users/User/Documents/Projects/FootballManagerCode/data/*'))
files_and_dirs = glob.glob(directory_path)

if files_and_dirs:
    latest_file = max(files_and_dirs, key=os.path.getctime)
    print("Latest file:", latest_file)
else:
    print("No files found in the specified directory.")

Latest file: /Users/User/Documents/Projects/FootballManagerCode/data\Birm782023.html


In [64]:
# read html file exported by FM 
rawdata_list = pd.read_html(latest_file, header=0, encoding="utf-8", keep_default_na=False)

In [65]:
# turn into a dataframe
df = rawdata_list[0]

For calculating speed, workrate and set piece metrics, I chose to use a geometric mean over an arithmetic mean, (df['Pac'] + df['Acc']) / 2. 

It offers advantages:

1. Balanced Attributes: The geometric mean is particularly sensitive to values that are significantly lower than others. This means if a player has a very high value in one attribute (e.g., Pac) but a low value in another (e.g., Acc), their overall Spd score won't be inflated. 
2. Reduced Skewness.
3. Non-linear scaling - a combination of two attributes might have a more significant impact than the sum of their individual effects. 

In [66]:
# Calculate speed, workrate and set piece scores
df['Spd'] = (df['Pac'] * df['Acc']) ** 0.5
df['Work'] = (df['Wor'] + df['Sta']) ** 0.5
df['SetPiece'] = (df['Jum'] + df['Bra']) ** 0.5

In [67]:
# Calculate the overall sweeper keeper support score using geometric mean

# Essential attributes
essential_product = np.power(df['Agi'] * df['Ref'], 1/2)
# Core attributes 
core_product = np.power(df['1v1'] * df['Ant'] * df['Cmd'] * df['Cnt'] * df['Kic'] * df['Pos'], 1/6)
# Secondary attributes
secondary_product = np.power(df['Acc'] * df['Aer'] * df['Cmp'] * df['Dec'] * df['Fir'] * df['Han'] * df['Pas'] * df['Thr'] * df['Vis'], 1/9)

df['sk_support'] = (essential_product * 5 + core_product * 3 + secondary_product * 1) / 9  # We divide by 9 because we're averaging three products
df['sk_support'] = df['sk_support'].round(1)

In [68]:
# Calculate the overall wingback support role score (wb)
df['wb_support'] = (
    # Essential attributes
    ((df['Wor'] + df['Acc'] + df['Pac'] + df['Sta']) * 5) +
    # Core attributes 
    ((df['Cro'] + df['Mar'] + df['Dri'] + df['Tck'] + df['OtB'] + df['Tea']) * 3) +
    # Secondary attributes
    ((df['Fir'] + df['Pas'] + df['Tec'] + df['Ant'] + df['Cnt'] + df['Dec'] + df['Pos'] + df['Agi'] + df['Bal']) * 1)
) / (5*4 + 3*6 + 1*9)  # The denominators are the sum of weights for each category

df['wb_support'] = df['wb_support'].round(1)

In [69]:
# Calculate the overall ball playing defender support role score (bpd_support)
df['bpd_support'] = (
    # Key attributes
    ((df['Hea'] + df['Mar'] + df['Pas'] + df['Tck'] + df['Cmp'] + 
      df['Pos'] + df['Jum'] + df['Str']) * 5) +
      
    # Desirable attributes 
    ((df['Fir'] + df['Tec'] + df['Agg'] + df['Ant'] + 
      df['Bra'] + df['Cnt'] + df['Dec'] + df['Vis'] + df['Pac']) * 3)
) / (5*8 + 3*9)  # The denominators are the sum of weights for each category

df['bpd_support'] = df['bpd_support'].round(1)

In [70]:
# Calculate the overall defensive midfielder score (dm_support)
df['dm_support'] = (
    # Essential attributes
    ((df['Wor'] + df['Pac']) * 5) +
    
    # Core attributes 
    ((df['Sta'] + df['Pas']) * 3) +
    
    # Secondary attributes
    ((df['Tck'] + df['Ant'] + df['Cnt'] + 
      df['Pos'] + df['Bal'] + df['Agi']) * 2) +
      
    # Tertiary attributes
    ((df['Tea'] + df['Fir'] + df['Mar'] + 
      df['Agg'] + df['Cmp'] + df['Dec'] + 
      df['Str']) * 1)
) / 35  # The denominator is the sum of weights for each category

df['dm_support'] = df['dm_support'].round(1)

In [71]:
# Calculate the overall box to box midfielder support role score (bbm_support)
df['bbm_support'] = (
    # Key attributes
    ((df['Pas'] + df['Tck'] + df['OtB'] + df['Pac'] + 
      df['Tea'] + df['Wor'] + df['Sta']) * 5) +
      
    # Secondary attributes 
    ((df['Dri'] + df['Fin'] + df['Fir'] + 
      df['Lon'] + df['Tec'] + df['Agg'] + 
      df['Ant'] + df['Cmp'] + df['Dec'] + 
      df['Pos'] + df['Acc'] + df['Bal'] + 
      df['Str']) * 3)
) / (5*6 + 3*14)  # The denominators are the sum of weights for each category

df['bbm_support'] = df['bbm_support'].round(1)

In [72]:
# Calculate the overall mezzala attacking role score (mezzala_atk)
df['mezzala_atk'] = (
    # Key attributes
    ((df['Dri'] + df['Pas'] + df['Tec'] + 
      df['Dec'] + df['OtB'] + df['Vis'] + 
      df['Wor'] + df['Acc']) * 5) +
      
    # Desirable attributes 
    ((df['Fin'] + df['Fir'] + df['Lon'] + 
      df['Ant'] + df['Cmp'] + df['Fla'] + 
      df['Bal'] + df['Sta']) * 3)
) / (5*8 + 3*8)  # The denominators are the sum of weights for each category

df['mezzala_atk'] = df['mezzala_atk'].round(1)

In [73]:
# Calculate the overall inside forward attacking role score (insideforward_atk)
df['insideforward_atk'] = (
    # Key attributes (highlighted green in the image)
    ((df['Dri'] + df['Fin'] + df['Fir'] + 
      df['Tec'] + df['OtB'] + df['Acc'] + 
      df['Pac']) * 5) +
      
    # Desirable attributes (highlighted blue in the image)
    ((df['Lon'] + df['Pas'] + df['Ant'] + 
      df['Cmp']) * 3)
) / (5*8 + 3*9)  # The denominators are the sum of weights for each category

df['insideforward_atk'] = df['insideforward_atk'].round(1)

In [74]:
# Calculate the overall inverted winger attacking role score (invertedwinger_atk)
df['invertedwinger_atk'] = (
    # Key attributes (highlighted green in the image)
    ((df['Dri'] + df['Pas'] + df['Fir'] + 
      df['Tec'] + df['Dec'] + df['OtB'] +
      df['Vis'] + df['Acc'] + df['Pac']) * 5) +
      
    # Desirable attributes (highlighted blue in the image)
    ((df['Cro'] + df['Lon'] + df['Ant'] + df['Cmp'] + 
      df['Fla'] + df['Vis'] + df['Agi'] + df['Sta']) * 3)
) / (5*10 + 3*8)  # The denominators are the sum of weights for each category

df['invertedwinger_atk'] = df['invertedwinger_atk'].round(1)


In [75]:
# Calculate the overall advanced forward attacking role score (advanceforward_atk)
df['advanceforward_atk'] = (
    # Key attributes (highlighted green in the image)
    ((df['Dri'] + df['Fin'] + df['Fir'] + 
      df['Tec'] + df['OtB'] + df['Cmp'] +
      df['Acc'] + df['Pac']) * 5) +
      
    # Desirable attributes (highlighted blue in the image)
    ((df['Pas'] + df['Ant'] + df['Dec'] + 
      df['Wor'] + df['Agi'] + df['Bal'] + df['Sta']) * 3)
) / (5*10 + 3*6)  # The denominators are the sum of weights for each category

df['advanceforward_atk'] = df['advanceforward_atk'].round(1)

In [76]:
df

,Inf,Name,Position,Nat,Age,Club,Transfer Value,Wage,Min AP,Min Fee Rls,...,SetPiece,sk_support,wb_support,bpd_support,dm_support,bbm_support,mezzala_atk,insideforward_atk,invertedwinger_atk,advanceforward_atk
0,Wnt,Luiz Gustavo,"DM, M (C)",BRA,35,,€0,-,-,-,...,5.567764,4.7,11.2,13.5,12.1,12.8,11.5,7.9,10.3,10.2
1,FrA,Ashkan Dejagah,AM (RLC),IRN,37,,€0,-,-,-,...,5.099020,4.2,10.4,11.3,10.8,11.7,12.8,8.3,11.6,10.6
2,FrA,Giedrius Arlauskis,GK,LTU,35,,€0,-,-,-,...,5.291503,12.1,9.3,9.9,10.6,9.5,8.6,5.2,7.9,8.0
3,FrA,Papiss Cissé,ST (C),SEN,38,,€0,-,-,-,...,4.690416,4.2,10.7,10.0,10.8,11.3,11.8,8.4,10.9,10.6
4,Wnt,Nikola Kalinić,ST (C),CRO,35,,€0,-,-,-,...,5.196152,4.8,11.1,10.8,11.4,12.4,12.6,8.6,11.1,11.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22524,Ama,Adileno Costa,ST (C),CPV,25,Belo Horizonte,€0,N/A,-,-,...,3.464102,3.6,4.9,3.7,3.9,4.7,4.8,4.9,5.2,5.9
22525,Ama,Levidson,M (C),CPV,27,Académica Porto Novo,€0,N/A,-,-,...,3.872983,4.0,6.8,5.7,7.0,6.4,6.5,4.6,6.5,6.2
22526,FrA,David Astolfi,D (C),ITA,25,,€0,-,-,-,...,3.741657,3.3,5.5,5.9,5.7,4.2,3.5,2.7,4.1,3.9
22527,FrA,Hassan Muhammad Sanusi,"AM (RL), ST (C)",NGA,23,,€0,-,-,-,...,2.449490,5.0,7.6,4.0,6.9,6.9,7.2,5.6,6.6,7.5


In [77]:
# builds squad dataframe using only columns that will be exported to HTML
squad = df[['Inf','Name','Age','Club','Transfer Value','Wage','Nat','Position','Personality','Media Handling','Left Foot', 'Right Foot','Spd','Jum','Str','Work','Height','sk_support','wb_support','bpd_support','dm_support','bbm_support', 'mezzala_atk', 'insideforward_atk', 'invertedwinger_atk', 'advanceforward_atk']]

In [78]:
# taken from here: https://www.thepythoncode.com/article/convert-pandas-dataframe-to-html-table-python
# creates a function to make a sortable html export

def generate_html(dataframe: pd.DataFrame):
    # get the table HTML from the dataframe
    table_html = dataframe.to_html(table_id="table", index=False)
    # construct the complete HTML with jQuery Data tables
    # You can disable paging or enable y scrolling on lines 20 and 21 respectively
    html = f"""
    <html>
    <header>
        <link href="https://cdn.datatables.net/1.11.5/css/jquery.dataTables.min.css" rel="stylesheet">
    </header>
    <body>
    {table_html}
    <script src="https://code.jquery.com/jquery-3.6.0.slim.min.js" integrity="sha256-u7e5khyithlIdTpu22PHhENmPcRdFiHRjhAuHcs05RI=" crossorigin="anonymous"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.11.5/js/jquery.dataTables.min.js"></script>
    <script>
        $(document).ready( function () {{
            $('#table').DataTable({{
                paging: true,
                order: [[12, 'desc']],
                // scrollY: 400,
            }});
        }});
    </script>
    </body>
    </html>
    """
    # return the html
    return html

In [79]:
# generates random file name for write-out of html file
# import uuid
# filename = str(uuid.uuid4()) + ".html"
# filename

In [80]:
# creates a sortable html export from the dataframe 'squad'

# html = generate_html(squad)
# open(filename, "w", encoding="utf-8").write(html)

In [81]:
# win_filename = os.path.join('/Users/User/Documents/Projects/FootballManagerCode/output/' + filename)

In [82]:
squad.to_excel("scout.xlsx")  

In [83]:
print(df.sort_values(by=['sk_support'], ascending=False)[['Name', 'sk_support']])

                     Name  sk_support
80          Renan Ribeiro        13.4
1367       Julio Iricíbar        13.1
107           Diego Alves        12.9
134           Joris Delle        12.9
60           Kiko Casilla        12.9
...                   ...         ...
20941     Ivan Stojanovic         2.3
20299       Way Chun Wang         2.3
16762     Fabiano Lorusso         2.2
17899         Phil Buxton         2.2
21600  Muhd Izman Rothman         2.2

[22529 rows x 2 columns]
